In [ ]:
#Code to remove EXIF data from images.
import glob
import os

dirs = glob.glob(os.path.join('./BlobStorage/test_data/*/*jpg'))

import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 


for file in dirs:
    img = mpimg.imread(file) 


import piexif
piexif.remove(img)

#Refer from here
#https://stackoverflow.com/questions/51219159/remove-exif-from-all-files-in-directory

In [ ]:
#Creating a subset of only 20 classes for test.
#import pandas as pd
import pickle
import shutil
import os

f = open("fasttext/clusterCenters.txt",'r')

lines = f.readlines()


line = lines[0].split()
print(line)
modelName = line[0]
classesNow = line[1:]
print(classesNow)
f.close()

data_dir = os.path.join(os.getcwd(),'BlobStorage')
test_data_dir = os.path.join(data_dir, 'test_data') # the categories need to be in folders
test_data_20_dir = os.path.join(data_dir, 'test_data_20') # the categories need to be in folders

for class_name in classesNow:
    shutil.copytree(test_data_dir+'/'+class_name,test_data_20_dir+'/'+class_name)




In [1]:
from keras.models import load_model
import os
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

data_dir = os.path.join(os.getcwd(),'BlobStorage')
test_data_dir = os.path.join(data_dir, 'test_data_20') # the categories need to be in folders
print(test_data_dir)

img_width,img_height = 224, 224
batch_size = 16

test_datagen = ImageDataGenerator(rescale=1. / 255) 

test_generator = test_datagen.flow_from_directory( 
        test_data_dir, 
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        class_mode='categorical',
        #class_mode=None,
        shuffle = False) 

test_generator.reset()

#Getting list of stored models in trained_models folder
#models_list = [l for l in os.listdir("trained_models") if l.endswith('.h5')]
#print(models_list)

#model = load_model("model_contextobject_4classes.h5")

filenames =  test_generator.filenames
nb_samples = len(filenames)

print('nb_samples '+str(nb_samples))
print(filenames[1])
#predictions = model.predict_generator(test_generator, steps=nb_samples)

Using TensorFlow backend.


/data/home/karan1234/notebooks/Karan-Paper/BlobStorage/test_data_20
Found 7530 images belonging to 20 classes.
nb_samples 7530
bag/000180340_original_1024x1024.jpg.jpg


In [2]:
import pandas as pd
import pickle

f = open("fasttext/clusterCenters.txt",'r')

lines = f.readlines()

for line in lines:
    line = line.split()
    print(line)
    modelName = line[0]
    classesNow = line[1:]
    
    #Keras sorts the list of classes used for training
    classesNow.sort()
    
    model = load_model('trained_models/'+modelName+'.h5')
    predictions = model.predict_generator(test_generator, steps=nb_samples/batch_size, verbose = 1)
    pred_df = pd.DataFrame(data=predictions, index=filenames, columns=classesNow)
    #print(pred_df.head())
    #print(pred_df.shape)
    pred_df.to_csv('predictions/all_categories/'+modelName+'.txt', header=True, index=True, sep=',',mode = 'w+')
    
    #Finding highest probability category
    pred_df['max_prob'] = pred_df.idxmax(axis=1)
    pred_df['max_prob'].to_csv('predictions/'+modelName+'.txt', header=True, index=True, sep=',',mode = 'w+')
    #pred_df.to_pickle('predictions/'+modelName+'.pkl') 

f.close()


W0404 22:01:25.096070 140410433156864 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0404 22:01:25.117036 140410433156864 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0404 22:01:25.142859 140410433156864 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



['model20', 'sunset', 'railway', 'river', 'shoulder', 'bike', 'painting', 'coin', 'coffee', 'championship', 'bag', 'football', 'photograph', 'scene', 'weather', 'shore', 'room', 'car', 'dress', 'roof', 'video']


W0404 22:01:25.449330 140410433156864 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0404 22:01:25.450126 140410433156864 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0404 22:01:25.450671 140410433156864 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0404 22:01:26.618477 140410433156864 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0404 22:01:26.71973

471/470 [==============================] - 84s 177ms/step


In [3]:
#Finding closest words to top predictions on testing set
import math
import pickle
from scipy.spatial import distance
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def scipy_distance(v, u):
    return distance.euclidean(v, u)

#Reading the fasttext dictionary populated at clustering phase
fastext_dict = pickle.load(open("fasttext/fastext_dict.pkl","rb"))
print(len(fastext_dict))
#print(fastext_dict.keys())
print(fastext_dict['car'])

total_classes = 379

dict_keys = list(fastext_dict.keys())

379
[-0.092271, -0.14855, -0.14696, 0.013, -0.40305, -0.31004, 0.1022, -0.42087, -0.22948, 0.12853, -0.34886, 0.072545, 0.081316, -0.13273, -0.011657, 0.18024, -0.14901, -0.090313, -0.12595, 0.35055, -0.094697, -0.094339, -0.47509, -0.37384, 0.063611, 0.056118, -0.32637, 0.22324, -0.2292, 0.24179, 0.017067, 0.24906, -0.20583, 0.14183, -0.20341, 0.18192, -0.22186, -0.10049, 0.02935, 0.039431, -0.39592, -0.27336, -0.027534, -0.14903, 0.044646, 0.2911, -0.06559, -0.21112, 0.51928, -0.1435, 0.096369, -0.16417, 0.11613, 0.16193, -0.15615, 0.038471, 0.33975, 0.60247, 0.18831, 0.2651, 0.21832, -0.0048556, 0.29734, 0.057363, -0.23942, -0.21652, -0.3096, 0.19925, 0.23067, -0.13709, -0.28048, 0.31534, 0.13813, -0.24841, 0.02902, 0.12314, -0.13154, 0.33097, -0.035216, 0.35946, 0.11113, 0.1922, -0.37032, -0.21848, -0.28887, 0.074491, -0.14468, -0.21976, 0.29048, 0.0011665, 0.0023941, -0.17978, -0.13347, 0.17614, 0.25683, -0.2448, -0.2018, 0.015822, -0.18401, -0.33955, -0.25707, -0.29338, -0.005028

In [ ]:
list(fastext_dict.keys())

In [4]:
#Generating the close words dictionary for each model
f = open("fasttext/clusterCenters.txt",'r')

lines = f.readlines()

for line in lines:
    
    line = line.split()
    print(line)
    modelName = line[0]
    model_k = int(modelName[5:])
    print(model_k)
    classesNow = line[1:]
    
    closeWords_Count = math.ceil(total_classes/model_k)
    print(closeWords_Count)
    
    closeWord_dict = {}
    
    for word in classesNow:
        distance_dict = {}
        
        for fast_word in dict_keys:
            dist = scipy_distance(fastext_dict[word],fastext_dict[fast_word])
            distance_dict[fast_word] = dist
            
        #sorted_distace_dict = {k: v for k, v in sorted(distance_dict.items(), key=lambda item: item[1],reverse = True)[:closeWords_Count+1]}
        closeWords_dict = {k: v for k, v in sorted(distance_dict.items(), key=lambda item: item[1])[:closeWords_Count+1]}
        
        #print(len(closeWords_dict))
        #closeWords_dict = take(closeWords_Count+1, sorted_distace_dict.items())
        closeWords_dict_keys = list(closeWords_dict.keys())
        
        closeWord_dict[word] = closeWords_dict_keys
    
    p = open(modelName+"_closeWord_dict.pkl","wb")
    pickle.dump(closeWord_dict,p)
    p.close()
           
    #pred_df = pd.read_csv('predictions/'+modelName+'.txt', header=True, index=True, sep=',')
f.close()


['model20', 'sunset', 'railway', 'river', 'shoulder', 'bike', 'painting', 'coin', 'coffee', 'championship', 'bag', 'football', 'photograph', 'scene', 'weather', 'shore', 'room', 'car', 'dress', 'roof', 'video']
20
19


In [ ]:

#closeWord_dict_list = [l for l in os.listdir("trained_models") if l.endswith('.pkl')]
#print(models_list)



closeWord_dict = pickle.load(open("model50_closeWord_dict.pkl","rb"))
print(len(closeWord_dict))
print(closeWord_dict.keys())

#print(closeWord_dict.keys())
closeWord_dict['water']


In [5]:
#Running final predictions
import numpy as np
import pandas as pd
import pickle

def prediction_check(y_pred,y_act):
    if y_act in y_pred:
        return 1
    else:
        return 0

f = open("fasttext/clusterCenters.txt",'r')

lines = f.readlines()

for line in lines:
    
    line = line.split()
    #print(line)
    modelName = line[0]
    print(modelName)
    #model_k = int(modelName[5:])
    #print(model_k)
    classesNow = line[1:]
    
    #Reading the predictions for each model
    pred_df = pd.read_csv('predictions/'+modelName+'.txt', header=0, sep=',')
    #print(pred_df.head)
    pred_df.columns = ['test_file','max_prob']
    pred_df['actual_label'] = pred_df.test_file.apply(lambda x: x.split('/')[0])
    #print(pred_df.columns)
    #print(pred_df.head())
    
    closeWord_dict = pickle.load(open(modelName+"_closeWord_dict.pkl","rb"))
    
    
    #closeWord_df = pd.DataFrame.from_dict(closeWord_dict, orient='index')
    #closeWord_df.columns = ['test_file','max_prob']
    #print(closeWord_df.columns)
    
    pred_df['guesses'] = pred_df['max_prob'].map(closeWord_dict)
    
    #pred_df['pred_check'] = pred_df.apply(lambda x: prediction_check(pred_df['guesses'],pred_df['actual_label']),axis = 1)
    
    pred_check = []
    
    #pred_df['pred_check'] = np.where(pred_df['actual_label'] in pred_df['guesses'],1,0)
    for index,row in pred_df.iterrows():
        if row['actual_label'] in row['guesses']:
            #print('actual label '+ row['actual_label'])
            #print('guesses '+ ' '.join(row['guesses']))
            pred_check.append(1)
        else:
            pred_check.append(0)
        
    pred_df['pred_check'] = pred_check
    
    total_right = pred_df['pred_check'].sum()
    print(total_right)
    
    total_rows = len(pred_df)
    print(total_rows)
    
    print('accuracy')
    print(total_right/total_rows)
    
    #print(pred_df.head())
    #df['combined'] = df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    
    #print(closeWord_df.head)

model20
471
7530
accuracy
0.06254980079681274
